In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta
# 판다스 엑셀 익스포트를 위한 라이브러리
import openpyxl


PROJECT = "ballosodeuk"
bq = bigquery.Client(project=PROJECT)


query = """
select a.*, survival_prob, is_churn, current_shoji, frequency, recency, monetary, grade, loyalty_segment, days_since_last_activity
from
  (
    select * from `dm.agg_user_category_rank`
    where power_1_2 is not null
  ) a
left join
  (
    select user_id, survival_prob, is_churn, loyalty_segment, days_since_last_activity
    from `dm.agg_user_churn_daily`
    where cur_date = current_date() - 1
  ) b on a.user_id = b.user_id
left join 
  (
    select *
    from `dm.agg_user_rfm_daily`
    where register_dt = current_date() -1
  ) c on a.user_id = c.user_id
"""

df = bq.query(query).to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [94]:
# df.to_excel('seg_power.xlsx', engine='openpyxl', encoding='utf-8-sig', index=False)

In [2]:
df.is_churn.value_counts()

False    4686
True     1668
Name: is_churn, dtype: Int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6388 entries, 0 to 6387
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   6388 non-null   object 
 1   latest_order_dt           6388 non-null   dbdate 
 2   total_purchase_count      6388 non-null   Int64  
 3   ranking_1_1               6388 non-null   object 
 4   power_1_1                 6388 non-null   float64
 5   ranking_1_2               6388 non-null   object 
 6   power_1_2                 6388 non-null   float64
 7   ranking_1_3               1259 non-null   object 
 8   power_1_3                 1259 non-null   float64
 9   ranking_1_4               341 non-null    object 
 10  power_1_4                 341 non-null    float64
 11  ranking_1_5               105 non-null    object 
 12  power_1_5                 105 non-null    float64
 13  ranking_2_1               6388 non-null   object 
 14  power_2_

In [4]:
# 전처리 및 null 처리
import re
import datetime
today = datetime.datetime.now().date()

df.latest_order_dt = pd.to_datetime(df.latest_order_dt)

df['latest_order_dt'] = (today - df.latest_order_dt.dt.date).dt.days
df.info()


power_cols = [col for col in df.columns.tolist() if col.startswith('power')]
ranking_cols = [col for col in df.columns.tolist() if col.startswith('ranking')]
df[power_cols] = df[power_cols].fillna(0)
df[power_cols] = df[power_cols].astype(float)

df['power_1_2'] = df['power_1_2'].fillna(0)
df['power_1_2'] = df['power_1_2'].astype(float)

df['current_shoji'] = df['current_shoji'].fillna(0)
df['current_shoji'] = df['current_shoji'].astype(float)

df['frequency'] = df['frequency'].fillna(0)
df['frequency'] = df['frequency'].astype(float)

df['is_churn'] = df['is_churn'].fillna(True)

df['recency'] = df['recency'].fillna(0)
df['recency'] = df['recency'].astype(float)

df['monetary'] = df['monetary'].fillna(0)
df['monetary'] = df['monetary'].astype(float)

df['grade'] = df['grade'].fillna("DEAD")
df['grade'] = df['grade'].astype(str)

df['current_shoji'] = df['current_shoji'].fillna(0)
df['current_shoji'] = df['current_shoji'].astype(float)

df['survival_prob'] = df['survival_prob'].fillna(0)
df['survival_prob'] = df['survival_prob'].astype(float)

df['loyalty_segment'] = df['loyalty_segment'].fillna("DEAD")
df['loyalty_segment'] = df['loyalty_segment'].astype(str)

df['days_since_last_activity'] = df['days_since_last_activity'].fillna(9999)
df['days_since_last_activity'] = df['days_since_last_activity'].astype(float)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6388 entries, 0 to 6387
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   6388 non-null   object 
 1   latest_order_dt           6388 non-null   int64  
 2   total_purchase_count      6388 non-null   Int64  
 3   ranking_1_1               6388 non-null   object 
 4   power_1_1                 6388 non-null   float64
 5   ranking_1_2               6388 non-null   object 
 6   power_1_2                 6388 non-null   float64
 7   ranking_1_3               1259 non-null   object 
 8   power_1_3                 1259 non-null   float64
 9   ranking_1_4               341 non-null    object 
 10  power_1_4                 341 non-null    float64
 11  ranking_1_5               105 non-null    object 
 12  power_1_5                 105 non-null    float64
 13  ranking_2_1               6388 non-null   object 
 14  power_2_

In [5]:
# 스파스 인코딩

power_cols = [col for col in df.columns.tolist() if col.startswith('power')]
ranking_cols = [col for col in df.columns.tolist() if col.startswith('ranking')]
tst = df[ranking_cols+power_cols].copy()

pivot_dfs = []

# ranking_1_x와 power_1_x 처리
for i in range(1, 6):  
    # NaN 값을 피벗 전에 처리
    temp_df = tst[[f'ranking_1_{i}', f'power_1_{i}']].copy()
    temp_df[f'ranking_1_{i}'] = temp_df[f'ranking_1_{i}'].fillna('unknown')  # 또는 다른 값으로 대체
    
    pivot_df = temp_df.melt(
        id_vars=[f'ranking_1_{i}'], 
        value_vars=[f'power_1_{i}']
    ).reset_index()\
    .pivot(
        index='index', 
        columns=f'ranking_1_{i}', 
        values='value'
    ).fillna(0)
    
    # unknown 컬럼 제거 (필요한 경우)
    if 'unknown' in pivot_df.columns:
        pivot_df = pivot_df.drop('unknown', axis=1)
    
    pivot_df.columns = [f'{col}_1_{i}' for col in pivot_df.columns]
    pivot_dfs.append(pivot_df)

# ranking_2_x와 power_2_x 처리도 동일하게 수정
for i in range(1, 11):
    temp_df = tst[[f'ranking_2_{i}', f'power_2_{i}']].copy()
    temp_df[f'ranking_2_{i}'] = temp_df[f'ranking_2_{i}'].fillna('unknown')
    
    pivot_df = temp_df.melt(
        id_vars=[f'ranking_2_{i}'], 
        value_vars=[f'power_2_{i}']
    ).reset_index()\
    .pivot(
        index='index', 
        columns=f'ranking_2_{i}', 
        values='value'
    ).fillna(0)
    
    if 'unknown' in pivot_df.columns:
        pivot_df = pivot_df.drop('unknown', axis=1)
        
    pivot_df.columns = [f'{col}_2_{i}' for col in pivot_df.columns]
    pivot_dfs.append(pivot_df)

# 모든 피벗 테이블을 index를 기준으로 결합
final_df = pd.concat(pivot_dfs, axis=1)

In [73]:
# 스파스 인코딩 (기존 코드)
power_cols = [col for col in df.columns.tolist() if col.startswith('power')]
ranking_cols = [col for col in df.columns.tolist() if col.startswith('ranking')]
tst = df[ranking_cols+power_cols].copy()

pivot_dfs = []

# ranking_1_x와 power_1_x 처리
for i in range(1, 6):  
    # NaN 값을 피벗 전에 처리
    temp_df = tst[[f'ranking_1_{i}', f'power_1_{i}']].copy()
    temp_df[f'ranking_1_{i}'] = temp_df[f'ranking_1_{i}'].fillna('unknown')  # 또는 다른 값으로 대체
    
    pivot_df = temp_df.melt(
        id_vars=[f'ranking_1_{i}'], 
        value_vars=[f'power_1_{i}']
    ).reset_index()\
    .pivot(
        index='index', 
        columns=f'ranking_1_{i}', 
        values='value'
    ).fillna(0)
    
    # unknown 컬럼 제거 (필요한 경우)
    if 'unknown' in pivot_df.columns:
        pivot_df = pivot_df.drop('unknown', axis=1)
    
    pivot_df.columns = [f'{col}_1_{i}' for col in pivot_df.columns]
    pivot_dfs.append(pivot_df)

# ranking_2_x와 power_2_x 처리도 동일하게 수정
for i in range(1, 11):
    temp_df = tst[[f'ranking_2_{i}', f'power_2_{i}']].copy()
    temp_df[f'ranking_2_{i}'] = temp_df[f'ranking_2_{i}'].fillna('unknown')
    
    pivot_df = temp_df.melt(
        id_vars=[f'ranking_2_{i}'], 
        value_vars=[f'power_2_{i}']
    ).reset_index()\
    .pivot(
        index='index', 
        columns=f'ranking_2_{i}', 
        values='value'
    ).fillna(0)
    
    if 'unknown' in pivot_df.columns:
        pivot_df = pivot_df.drop('unknown', axis=1)
        
    pivot_df.columns = [f'{col}_2_{i}' for col in pivot_df.columns]
    pivot_dfs.append(pivot_df)

# 모든 피벗 테이블을 index를 기준으로 결합
final_df = pd.concat(pivot_dfs, axis=1)

# 동일한 카테고리를 하나로 합치는 코드
# 컬럼 이름에서 접미사(_1_1, _2_3 등)를 제거한 기본 카테고리명을 추출
def extract_category_name(col_name):
    # '_1_1', '_2_3' 등의 접미사를 제거하고 기본 카테고리명만 반환
    if '_1_' in col_name or '_2_' in col_name:
        # 마지막 '_' 이전까지의 문자열을 카테고리명으로 간주
        parts = col_name.split('_')
        if len(parts) >= 3:
            # 마지막 두 부분(숫자)을 제외한 나머지를 결합
            return '_'.join(parts[:-2])
    return col_name

# 동일한 카테고리를 그룹화
category_groups = {}
for col in final_df.columns:
    base_category = extract_category_name(col)
    if base_category not in category_groups:
        category_groups[base_category] = []
    category_groups[base_category].append(col)

# 동일한 카테고리의 컬럼들을 합쳐서 새로운 데이터프레임 생성
merged_df = pd.DataFrame(index=final_df.index)

for category, columns in category_groups.items():
    # 같은 카테고리의 컬럼들을 합산
    merged_df[category] = final_df[columns].sum(axis=1)

# merged_df가 최종 결과물이 됩니다
print(f"원래 컬럼 수: {final_df.shape[1]}")
print(f"병합 후 컬럼 수: {merged_df.shape[1]}")
print(f"병합된 카테고리 목록: {merged_df.columns.tolist()}")

원래 컬럼 수: 639
병합 후 컬럼 수: 102
병합된 카테고리 목록: ['가구-인테리어', '디지털-가전', '생활-건강', '스포츠-레저', '식품', '출산-육아', '패션의류', '패션잡화', '화장품-미용', '가구-인테리어(DIY자재-용품)', '가구-인테리어(거실가구)', '가구-인테리어(수납가구)', '가구-인테리어(수예)', '가구-인테리어(인테리어소품)', '가구-인테리어(침구단품)', '가구-인테리어(카페트-러그)', '가구-인테리어(홈데코)', '디지털-가전(PC액세서리)', '디지털-가전(계절가전)', '디지털-가전(노트북)', '디지털-가전(생활가전)', '디지털-가전(음향가전)', '디지털-가전(이미용가전)', '디지털-가전(주방가전)', '디지털-가전(주변기기)', '디지털-가전(휴대폰악세서리)', '생활-건강(건강관리용품)', '생활-건강(건강측정용품)', '생활-건강(공구)', '생활-건강(구강위생용품)', '생활-건강(문구-사무용품)', '생활-건강(발건강용품)', '생활-건강(생활용품)', '생활-건강(세탁용품)', '생활-건강(수납-정리용품)', '생활-건강(안마용품)', '생활-건강(애완)', '생활-건강(온열-찜질용품)', '생활-건강(욕실용품)', '생활-건강(의료용품)', '생활-건강(자동차용품)', '생활-건강(주방용품)', '생활-건강(청소용품)', '스포츠-레저(골프)', '스포츠-레저(기타스포츠용품)', '스포츠-레저(등산)', '스포츠-레저(스포츠액세서리)', '스포츠-레저(캠핑)', '스포츠-레저(헬스)', '식품(가공식품)', '식품(건강식품)', '식품(과자)', '식품(김치)', '식품(냉동-간편조리식품)', '식품(농산물)', '식품(다이어트식품)', '식품(반찬)', '식품(수산)', '식품(음료)', '식품(축산)', '출산-육아(물티슈)', '출산-육아(위생-건강용품)', '패션의류(남성언더웨어-잠옷)', '패션의류(남성의류)', '패션의류(여성언더웨어-잠옷)', '패션의류(여성의류)', '

<ipython-input-73-95ef1126ef6a>:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[category] = final_df[columns].sum(axis=1)
<ipython-input-73-95ef1126ef6a>:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[category] = final_df[columns].sum(axis=1)


In [74]:
merged_df.head()

,가구-인테리어,디지털-가전,생활-건강,스포츠-레저,식품,출산-육아,패션의류,패션잡화,화장품-미용,가구-인테리어(DIY자재-용품),가구-인테리어(거실가구),가구-인테리어(수납가구),가구-인테리어(수예),가구-인테리어(인테리어소품),가구-인테리어(침구단품),가구-인테리어(카페트-러그),가구-인테리어(홈데코),디지털-가전(PC액세서리),디지털-가전(계절가전),디지털-가전(노트북),디지털-가전(생활가전),디지털-가전(음향가전),디지털-가전(이미용가전),디지털-가전(주방가전),디지털-가전(주변기기),디지털-가전(휴대폰악세서리),생활-건강(건강관리용품),생활-건강(건강측정용품),생활-건강(공구),생활-건강(구강위생용품),생활-건강(문구-사무용품),생활-건강(발건강용품),생활-건강(생활용품),생활-건강(세탁용품),생활-건강(수납-정리용품),생활-건강(안마용품),생활-건강(애완),생활-건강(온열-찜질용품),생활-건강(욕실용품),생활-건강(의료용품),생활-건강(자동차용품),생활-건강(주방용품),생활-건강(청소용품),스포츠-레저(골프),스포츠-레저(기타스포츠용품),스포츠-레저(등산),스포츠-레저(스포츠액세서리),스포츠-레저(캠핑),스포츠-레저(헬스),식품(가공식품),식품(건강식품),식품(과자),식품(김치),식품(냉동-간편조리식품),식품(농산물),식품(다이어트식품),식품(반찬),식품(수산),식품(음료),식품(축산),출산-육아(물티슈),출산-육아(위생-건강용품),패션의류(남성언더웨어-잠옷),패션의류(남성의류),패션의류(여성언더웨어-잠옷),패션의류(여성의류),패션잡화(모자),패션잡화(양말),패션잡화(여성가방),패션잡화(여성신발),패션잡화(여행용가방-소품),패션잡화(장갑),패션잡화(패션소품),패션잡화(헤어액세서리),화장품-미용(남성화장품),화장품-미용(마스크-팩),화장품-미용(바디케어),화장품-미용(베이스메이크업),화장품-미용(뷰티소품),화장품-미용(색조메이크업),화장품-미용(선케어),화장품-미용(스킨케어),화장품-미용(클렌징),화장품-미용(향수),화장품-미용(헤어스타일링),화장품-미용(헤어케어),가구-인테리어(서재-사무용가구),가구-인테리어(솜류),가구-인테리어(주방가구),디지털-가전(영상가전),생활-건강(재활운동용품),스포츠-레저(배드민턴),스포츠-레저(수영),스포츠-레저(자전거),식품(아이스크림-빙수),출산-육아(안전용퓸),출산-육아(완구-매트),출산-육아(외출용품),패션잡화(남성신발),패션잡화(신발용품),패션잡화(주얼리),패션잡화(지갑)
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.500000,0.0,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.500000,0.5,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.333333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.400000,0.0,0.600000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.571429,0.0,0.285714,0.0,0.0,0.000000,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.142857,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_list = df[['depth1_list','depth1plus_list', 'depth3_list']]
df_num = df[['user_id','latest_order_dt', 'total_purchase_count','survival_prob', 'is_churn',
       'current_shoji', 'frequency', 'recency', 'monetary', 'grade',
       'loyalty_segment', 'days_since_last_activity']]



In [7]:
df_num_sp = df_num.merge(final_df, left_index=True, right_index=True)
df_num_sp.grade.value_counts()

GOLD      2674
VIP       2014
SILVER     663
DEAD       555
IRON       482
Name: grade, dtype: int64

In [8]:
df_num_sp = df_num_sp.query("grade != 'DEAD'")
df_num_sp.head()

,user_id,latest_order_dt,total_purchase_count,survival_prob,is_churn,current_shoji,frequency,recency,monetary,grade,...,식품(축산)_2_10,패션의류(여성언더웨어-잠옷)_2_10,패션의류(여성의류)_2_10,패션잡화(양말)_2_10,화장품-미용(바디케어)_2_10,화장품-미용(색조메이크업)_2_10,화장품-미용(스킨케어)_2_10,화장품-미용(클렌징)_2_10,화장품-미용(헤어스타일링)_2_10,화장품-미용(헤어케어)_2_10
0,045696e7-11a9-4b00-9bee-abb3c1a6ff40,177,2,0.026767,False,421.0,1.0,176.0,21800.0,IRON,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,804bf7e9-6613-4d60-95a8-4ff7a909f803,177,2,0.000000,True,481.0,1.0,176.0,52700.0,IRON,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5d6e166d-9733-4e3a-af48-598e3052c461,177,3,0.000000,True,61.0,1.0,176.0,40600.0,IRON,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6156672b-959c-4369-a72a-f8d9898ffee7,177,7,0.839796,False,438.0,2.0,176.0,82800.0,GOLD,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,daacaf36-f529-4aeb-83cd-f48d41875881,38,23,0.825509,False,2441.0,15.0,37.0,604600.0,VIP,...,0.0,0.0,0.042621,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 모델

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import NMF
from sklearn.cluster import AgglomerativeClustering
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import skfuzzy as fuzz

pd.options.display.max_columns = None
# 모든 행과 열을 표시하도록 pandas 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [10]:
# 인코딩딩
le = LabelEncoder()
sc = StandardScaler()

grade_order = {
    'IRON': 0,    # 가장 낮은 등급
    'SILVER': 1,
    'GOLD': 2,
    'VIP': 3,     # 가장 높은 등급
}
segment_order = {
    "최상위 충성 유저" : 6,
    "장기 충성 유저" : 5,
    "충성 유저" : 4,
    "안정기 유저" : 3,
    "정착기 유저" : 2,
    "적응기 유저" : 1,
    "초기 유저" : 0,
}
# 순서대로 매핑
df_num_sp['grade_encoded'] = df_num_sp['grade'].map(grade_order)
df_num_sp['segment_encoded'] = df_num_sp['loyalty_segment'].map(segment_order)
df_num_sp['is_churn'] = df_num_sp.is_churn.map({True: 1, False: 0})

In [32]:
# 처리된 카테고리컬럼 드롭 & 유저아이디 인덱싱
df_num_sp.drop(['grade','loyalty_segment','latest_order_dt'],axis=1,inplace=True)
df_num_sp.set_index('user_id',inplace=True)

In [ ]:
#  'total_purchase_count' : 총 누적 구매 수 (int)
#  'survival_prob' : 현재 유저의 계산된 생존 확률 (float)
#  'is_churn' : 유저가 이탈했는지 여부. (1,0)
#  'current_shoji' : 현재 유저가 소유한 재산 (int)
#  'frequency' : 총 누적 구매 일 수(int)
#  'recency' : 오늘로부터 최근 구매 일의 기간 (int)
#  'monetary' : 누적 매출 (int)
#  'days_since_last_activity' : 오늘로부터 최근 활동 일의 기간 (int)
#  '가구-인테리어_1_1' : 유저가 구매한 상품의 카테고리 중, 계산된 선호도 (one hot encoded)
#  '디지털-가전_1_1' : 이하 내용 다 같음.
# ...

### 카테고리
NMF와 Fuzzy C-Means 조합은 매우 좋은 접근법입니다. 카테고리 개수가 45개와 594개로 많기 때문에 NMF를 통한 차원 축소는 효과적인 전략입니다. NMF(Non-negative Matrix Factorization)는 다음과 같은 장점이 있습니다:


음수가 없는 특성을 유지하므로 카테고리 선호도 데이터와 같은 양수 값에 적합합니다
원래 특성(카테고리)의 의미를 어느 정도 보존하면서 차원을 축소합니다
각 구성요소가 관련 카테고리들의 조합으로 해석될 수 있어 군집의 의미 파악에 도움이 됩니다

In [46]:
category_data

[['가구-인테리어_1_1',
  '디지털-가전_1_1',
  '생활-건강_1_1',
  '스포츠-레저_1_1',
  '식품_1_1',
  '출산-육아_1_1',
  '패션의류_1_1',
  '패션잡화_1_1',
  '화장품-미용_1_1',
  '가구-인테리어_1_2',
  '디지털-가전_1_2',
  '생활-건강_1_2',
  '스포츠-레저_1_2',
  '식품_1_2',
  '출산-육아_1_2',
  '패션의류_1_2',
  '패션잡화_1_2',
  '화장품-미용_1_2',
  '가구-인테리어_1_3',
  '디지털-가전_1_3',
  '생활-건강_1_3',
  '스포츠-레저_1_3',
  '식품_1_3',
  '출산-육아_1_3',
  '패션의류_1_3',
  '패션잡화_1_3',
  '화장품-미용_1_3',
  '가구-인테리어_1_4',
  '디지털-가전_1_4',
  '생활-건강_1_4',
  '스포츠-레저_1_4',
  '식품_1_4',
  '출산-육아_1_4',
  '패션의류_1_4',
  '패션잡화_1_4',
  '화장품-미용_1_4',
  '가구-인테리어_1_5',
  '디지털-가전_1_5',
  '생활-건강_1_5',
  '스포츠-레저_1_5',
  '식품_1_5',
  '출산-육아_1_5',
  '패션의류_1_5',
  '패션잡화_1_5',
  '화장품-미용_1_5',
  '가구-인테리어(DIY자재-용품)_2_1',
  '가구-인테리어(거실가구)_2_1',
  '가구-인테리어(수납가구)_2_1',
  '가구-인테리어(수예)_2_1',
  '가구-인테리어(인테리어소품)_2_1',
  '가구-인테리어(침구단품)_2_1',
  '가구-인테리어(카페트-러그)_2_1',
  '가구-인테리어(홈데코)_2_1',
  '디지털-가전(PC액세서리)_2_1',
  '디지털-가전(계절가전)_2_1',
  '디지털-가전(노트북)_2_1',
  '디지털-가전(생활가전)_2_1',
  '디지털-가전(음향가전)_2_1',
  '디지털-가전(이미용가전)_2_1',
  

In [52]:
# 카테고리 차원축소
cat_1 = [col for col in df_num_sp.columns if '_1_' in col]
cat_2 = [col for col in df_num_sp.columns if '_2_' in col]
print(len(cat_1), len(cat_2))
category_data = df_num_sp[cat_1 + cat_2]

n_components = 15
nmf = NMF(n_components=n_components, random_state=42)
reduced_data = nmf.fit_transform(category_data)

print('n-comp shape : ',reduced_data.shape)

# Fuzzy C-Means
n_clusters = 5
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    reduced_data.T,
    n_clusters,
    2,
    error = 0.005,
    maxiter = 1000
)

cluster_membership = np.argmax(u, axis=0)
df_num_sp['cluster'] = cluster_membership


45 594
n-comp shape :  (5833, 15)


In [59]:
df_num_sp.query("cluster == 2")

,total_purchase_count,survival_prob,is_churn,current_shoji,frequency,recency,monetary,days_since_last_activity,가구-인테리어_1_1,디지털-가전_1_1,생활-건강_1_1,스포츠-레저_1_1,식품_1_1,출산-육아_1_1,패션의류_1_1,패션잡화_1_1,화장품-미용_1_1,가구-인테리어_1_2,디지털-가전_1_2,생활-건강_1_2,스포츠-레저_1_2,식품_1_2,출산-육아_1_2,패션의류_1_2,패션잡화_1_2,화장품-미용_1_2,가구-인테리어_1_3,디지털-가전_1_3,생활-건강_1_3,스포츠-레저_1_3,식품_1_3,출산-육아_1_3,패션의류_1_3,패션잡화_1_3,화장품-미용_1_3,가구-인테리어_1_4,디지털-가전_1_4,생활-건강_1_4,스포츠-레저_1_4,식품_1_4,출산-육아_1_4,패션의류_1_4,패션잡화_1_4,화장품-미용_1_4,가구-인테리어_1_5,디지털-가전_1_5,생활-건강_1_5,스포츠-레저_1_5,식품_1_5,출산-육아_1_5,패션의류_1_5,패션잡화_1_5,화장품-미용_1_5,가구-인테리어(DIY자재-용품)_2_1,가구-인테리어(거실가구)_2_1,가구-인테리어(수납가구)_2_1,가구-인테리어(수예)_2_1,가구-인테리어(인테리어소품)_2_1,가구-인테리어(침구단품)_2_1,가구-인테리어(카페트-러그)_2_1,가구-인테리어(홈데코)_2_1,디지털-가전(PC액세서리)_2_1,디지털-가전(계절가전)_2_1,디지털-가전(노트북)_2_1,디지털-가전(생활가전)_2_1,디지털-가전(음향가전)_2_1,디지털-가전(이미용가전)_2_1,디지털-가전(주방가전)_2_1,디지털-가전(주변기기)_2_1,디지털-가전(휴대폰악세서리)_2_1,생활-건강(건강관리용품)_2_1,생활-건강(건강측정용품)_2_1,생활-건강(공구)_2_1,생활-건강(구강위생용품)_2_1,생활-건강(문구-사무용품)_2_1,생활-건강(발건강용품)_2_1,생활-건강(생활용품)_2_1,생활-건강(세탁용품)_2_1,생활-건강(수납-정리용품)_2_1,생활-건강(안마용품)_2_1,생활-건강(애완)_2_1,생활-건강(온열-찜질용품)_2_1,생활-건강(욕실용품)_2_1,생활-건강(의료용품)_2_1,생활-건강(자동차용품)_2_1,생활-건강(주방용품)_2_1,생활-건강(청소용품)_2_1,스포츠-레저(골프)_2_1,스포츠-레저(기타스포츠용품)_2_1,스포츠-레저(등산)_2_1,스포츠-레저(스포츠액세서리)_2_1,스포츠-레저(캠핑)_2_1,스포츠-레저(헬스)_2_1,식품(가공식품)_2_1,식품(건강식품)_2_1,식품(과자)_2_1,식품(김치)_2_1,식품(냉동-간편조리식품)_2_1,식품(농산물)_2_1,식품(다이어트식품)_2_1,식품(반찬)_2_1,식품(수산)_2_1,식품(음료)_2_1,식품(축산)_2_1,출산-육아(물티슈)_2_1,출산-육아(위생-건강용품)_2_1,패션의류(남성언더웨어-잠옷)_2_1,패션의류(남성의류)_2_1,패션의류(여성언더웨어-잠옷)_2_1,패션의류(여성의류)_2_1,패션잡화(모자)_2_1,패션잡화(양말)_2_1,패션잡화(여성가방)_2_1,패션잡화(여성신발)_2_1,패션잡화(여행용가방-소품)_2_1,패션잡화(장갑)_2_1,패션잡화(패션소품)_2_1,패션잡화(헤어액세서리)_2_1,화장품-미용(남성화장품)_2_1,화장품-미용(마스크-팩)_2_1,화장품-미용(바디케어)_2_1,화장품-미용(베이스메이크업)_2_1,화장품-미용(뷰티소품)_2_1,화장품-미용(색조메이크업)_2_1,화장품-미용(선케어)_2_1,화장품-미용(스킨케어)_2_1,화장품-미용(클렌징)_2_1,화장품-미용(향수)_2_1,화장품-미용(헤어스타일링)_2_1,화장품-미용(헤어케어)_2_1,가구-인테리어(DIY자재-용품)_2_2,가구-인테리어(거실가구)_2_2,가구-인테리어(서재-사무용가구)_2_2,가구-인테리어(솜류)_2_2,가구-인테리어(수납가구)_2_2,가구-인테리어(인테리어소품)_2_2,가구-인테리어(주방가구)_2_2,가구-인테리어(침구단품)_2_2,가구-인테리어(카페트-러그)_2_2,가구-인테리어(홈데코)_2_2,디지털-가전(PC액세서리)_2_2,디지털-가전(계절가전)_2_2,디지털-가전(생활가전)_2_2,디지털-가전(영상가전)_2_2,디지털-가전(음향가전)_2_2,디지털-가전(이미용가전)_2_2,디지털-가전(주방가전)_2_2,디지털-가전(휴대폰악세서리)_2_2,생활-건강(건강관리용품)_2_2,생활-건강(건강측정용품)_2_2,생활-건강(공구)_2_2,생활-건강(구강위생용품)_2_2,생활-건강(문구-사무용품)_2_2,생활-건강(발건강용품)_2_2,생활-건강(생활용품)_2_2,생활-건강(세탁용품)_2_2,생활-건강(수납-정리용품)_2_2,생활-건강(안마용품)_2_2,생활-건강(애완)_2_2,생활-건강(욕실용품)_2_2,생활-건강(의료용품)_2_2,생활-건강(자동차용품)_2_2,생활-건강(재활운동용품)_2_2,생활-건강(주방용품)_2_2,생활-건강(청소용품)_2_2,스포츠-레저(골프)_2_2,스포츠-레저(기타스포츠용품)_2_2,스포츠-레저(등산)_2_2,스포츠-레저(배드민턴)_2_2,스포츠-레저(수영)_2_2,스포츠-레저(스포츠액세서리)_2_2,스포츠-레저(자전거)_2_2,스포츠-레저(캠핑)_2_2,식품(가공식품)_2_2,식품(건강식품)_2_2,식품(과자)_2_2,식품(김치)_2_2,식품(냉동-간편조리식품)_2_2,식품(농산물)_2_2,식품(다이어트식품)_2_2,식품(반찬)_2_2,식품(수산)_2_2,식품(아이스크림-빙수)_2_2,식품(음료)_2_2,식품(축산)_2_2,출산-육아(물티슈)_2_2,출산-육아(안전용퓸)_2_2,출산-육아(완구-매트)_2_2,출산-육아(외출용품)_2_2,출산-육아(위생-건강용품)_2_2,패션의류(남성언더웨어-잠옷)_2_2,패션의류(남성의류)_2_2,패션의류(여성언더웨어-잠옷)_2_2,패션의류(여성의류)_2_2,패션잡화(남성신발)_2_2,패션잡화(모자)_2_2,패션잡화(신발용품)_2_2,패션잡화(양말)_2_2,패션잡화(여성가방)_2_2,패션잡화(여성신발)_2_2,패션잡화(장갑)_2_2,패션잡화(주얼리)_2_2,패션잡화(지갑)_2_2,패션잡화(패션소품)_2_2,패션잡화(헤어액세서리)_2_2,화장품-미용(남성화장품)_2_2,화장품-미용(마스크-팩)_2_2,화장품-미용(바디케어)_2_2,화장품-미용(베이스메이크업)_2_2,화장품-미용(뷰티소품)_2_2,화장품-미용(색조메이크업)_2_2,화장품-미용(선케어)_2_2,화장품-미용(스킨케어)_2_2,화장품-미용(클렌징)_2_2,화장품-미용(향수)_2_2,화장품-미용(헤어스타일링)_2_2,화장품-미용(헤어케어)_2_2,가구-인테리어(DIY자재-용품)_2_3,가구-인테리어(거실가구)_2_3,가구-인테리어(서재-사무용가구)_2_3,가구-인테리어(침구단품)_2_3,가구-인테리어(카페트-러그)_2_3,가구-인테리어(홈데코)_2_3,디지털-가전(PC액세서리)_2_3,디지털-가전(계절가전)_2_3,디지털-가전(생활가전)_2_3,디지털-가전(음향가전)_2_3,디지털-가전(이미용가전)_2_3,디지털-가전(주방가전)_2_3,디지털-가전(휴대폰악세서리)_2_3,생활-건강(건강관리용품)_2_3,생활-건강(건강측정용품)_2_3,생활-건강(공구)_2_3,생활-건강(구강위생용품)_2_3,생활-건강(문구-사무용품)_2_3,생활-건강(발건강용품)_2_3,생활-건강(생활용품)_2_3,생활-건강(세탁용품)_2_3,생활-건강(수납-정리용품)_2_3,생활-건강(안마용품)_2_3,생활-건강(애완)_2_3,생활-건강(욕실용품)_2_3,생활-건강(의료용품)_2_3,생활-건강(자동차용품)_2_3,생활-건강(주방용품)_2_3,생활-건강(청소용품)_2_3,스포츠-레저(골프)_2_3,스포츠-레저(등산)_2_3,스포츠-레저(자전거)_2_3,스포츠-레저(캠핑)_2_3,식품(가공식품)_2_3,식품(건강식품)_2_3,식품(과자)_2_3,식품(김치)_2_3,식품(냉동-간편조리식품)_2_3,식품(농산물)_2_3,식품(다이어트식품)_2_3,식품(반찬)_2_3,식품(수산)_2_3,식품(아이스크림-빙수)_2_3,식품(음료)_2_3,식품(축산)_2_3,출산-육아(물티슈)_2_3,출산-육아(안전용퓸)_2_3,출

In [56]:
df_num_sp.groupby('cluster').agg({
    'survival_prob':'count',
    
})

,survival_prob
cluster,
0,3
1,5
2,2896
3,1
4,2928


#### 군집 프로파일링

In [63]:
# 각 NMF 구성요소에 가장 큰 영향을 미치는 원본 카테고리 확인
all_category_columns = cat_1 + cat_2
component_interpretations = {}

for i in range(n_components):
    component = nmf.components_[i]
    top_indices = component.argsort()[::-1][:10]  # 상위 10개 카테고리
    top_categories = [all_category_columns[j] for j in top_indices]
    top_weights = component[top_indices]
    
    # 구성요소 이름 생성 (상위 3개 카테고리 기반)
    component_name = " + ".join([cat.split('_')[0] for cat in top_categories[:3]])
    component_interpretations[i] = {
        "name": component_name,
        "top_categories": list(zip(top_categories, top_weights))
    }
    
    print(f"\n구성요소 {i}: {component_name}")
    for cat, weight in zip(top_categories, top_weights):
        print(f"  - {cat.split('_')[0]}: {weight:.4f}")


구성요소 0: 식품 + 식품(가공식품) + 패션의류
  - 식품: 3.0798
  - 식품(가공식품): 2.9537
  - 패션의류: 0.1172
  - 가구-인테리어: 0.0707
  - 식품(농산물): 0.0674
  - 생활-건강(주방용품): 0.0610
  - 생활-건강: 0.0557
  - 패션의류(남성언더웨어-잠옷): 0.0362
  - 식품(냉동-간편조리식품): 0.0361
  - 패션의류(여성언더웨어-잠옷): 0.0329

구성요소 1: 생활-건강(주방용품) + 생활-건강 + 생활-건강
  - 생활-건강(주방용품): 4.1195
  - 생활-건강: 0.4600
  - 생활-건강: 0.1360
  - 식품(가공식품): 0.1295
  - 식품(농산물): 0.1265
  - 식품(냉동-간편조리식품): 0.0590
  - 화장품-미용(헤어케어): 0.0420
  - 식품(음료): 0.0408
  - 식품(가공식품): 0.0383
  - 식품(수산): 0.0349

구성요소 2: 화장품-미용 + 화장품-미용(바디케어) + 화장품-미용(헤어케어)
  - 화장품-미용: 3.9678
  - 화장품-미용(바디케어): 0.5102
  - 화장품-미용(헤어케어): 0.3526
  - 화장품-미용(클렌징): 0.3398
  - 화장품-미용(마스크-팩): 0.3072
  - 화장품-미용(선케어): 0.2743
  - 화장품-미용(스킨케어): 0.1478
  - 식품: 0.1411
  - 패션의류: 0.1307
  - 화장품-미용(남성화장품): 0.1092

구성요소 3: 생활-건강 + 생활-건강(주방용품) + 생활-건강(생활용품)
  - 생활-건강: 4.0057
  - 생활-건강(주방용품): 2.5183
  - 생활-건강(생활용품): 0.1745
  - 패션의류: 0.1136
  - 가구-인테리어: 0.1117
  - 생활-건강(건강관리용품): 0.1006
  - 생활-건강(청소용품): 0.0895
  - 생활-건강(건강측정용품): 0.0676
  - 생활-건강(구

In [64]:
# 각 군집의 NMF 구성요소 평균 값 계산
cluster_profiles = []
for cluster_id in range(n_clusters):
    mask = cluster_membership == cluster_id
    cluster_size = np.sum(mask)
    
    # 클러스터에 속한 사용자의 평균 NMF 구성요소 값
    cluster_nmf_profile = reduced_data[mask].mean(axis=0)
    
    # 주요 NMF 구성요소 파악 (상위 3개)
    top_components = cluster_nmf_profile.argsort()[::-1][:3]
    
    # 결과 저장
    cluster_info = {
        "cluster_id": cluster_id,
        "size": cluster_size,
        "percentage": cluster_size / len(reduced_data) * 100,
        "top_components": [(i, component_interpretations[i]["name"]) for i in top_components],
        "nmf_profile": cluster_nmf_profile
    }
    cluster_profiles.append(cluster_info)
    
    # 군집 정보 출력
    print(f"\n군집 {cluster_id}:")
    print(f"사용자 수: {cluster_size} ({cluster_size / len(reduced_data) * 100:.1f}%)")
    print("주요 관심 카테고리 그룹:")
    for comp_id, comp_name in cluster_info["top_components"]:
        print(f"  - {comp_name} (가중치: {cluster_nmf_profile[comp_id]:.4f})")


군집 0:
사용자 수: 3 (0.1%)
주요 관심 카테고리 그룹:
  - 생활-건강 + 생활-건강(주방용품) + 생활-건강(생활용품) (가중치: 0.1052)
  - 패션잡화 + 패션잡화(양말) + 패션의류 (가중치: 0.0642)
  - 식품 + 식품(음료) + 식품(가공식품) (가중치: 0.0032)

군집 1:
사용자 수: 5 (0.1%)
주요 관심 카테고리 그룹:
  - 생활-건강 + 생활-건강(주방용품) + 생활-건강(생활용품) (가중치: 0.0776)
  - 식품 + 식품(가공식품) + 패션의류 (가중치: 0.0348)
  - 디지털-가전 + 디지털-가전(주방가전) + 디지털-가전(생활가전) (가중치: 0.0079)

군집 2:
사용자 수: 2896 (49.6%)
주요 관심 카테고리 그룹:
  - 식품 + 식품(음료) + 식품(가공식품) (가중치: 0.0495)
  - 생활-건강 + 생활-건강(생활용품) + 가구-인테리어 (가중치: 0.0416)
  - 화장품-미용 + 화장품-미용(바디케어) + 화장품-미용(헤어케어) (가중치: 0.0333)

군집 3:
사용자 수: 1 (0.0%)
주요 관심 카테고리 그룹:
  - 생활-건강 + 생활-건강(주방용품) + 생활-건강(생활용품) (가중치: 0.0931)
  - 패션잡화 + 패션잡화(양말) + 패션의류 (가중치: 0.0072)
  - 디지털-가전 + 디지털-가전(주방가전) + 디지털-가전(생활가전) (가중치: 0.0069)

군집 4:
사용자 수: 2928 (50.2%)
주요 관심 카테고리 그룹:
  - 식품 + 식품(농산물) + 식품(수산) (가중치: 0.0710)
  - 식품 + 식품(음료) + 패션의류 (가중치: 0.0493)
  - 식품 + 식품(건강식품) + 패션의류 (가중치: 0.0463)


In [65]:
# 각 군집의 원본 카테고리 선호도 평균
for cluster_id in range(n_clusters):
    mask = cluster_membership == cluster_id
    
    # 원본 카테고리 데이터
    cluster_cat_profile = category_data.loc[mask].mean()
    
    # 상위 15개 선호 카테고리
    top_categories = cluster_cat_profile.sort_values(ascending=False).head(15)
    
    print(f"\n군집 {cluster_id} 상위 선호 카테고리:")
    for cat, score in zip(top_categories.index, top_categories.values):
        print(f"  - {cat.split('_')[0]}: {score:.4f}")


군집 0 상위 선호 카테고리:
  - 생활-건강: 0.4823
  - 패션잡화(양말): 0.1831
  - 패션잡화: 0.1831
  - 패션의류(남성언더웨어-잠옷): 0.1680
  - 패션의류: 0.1680
  - 가구-인테리어: 0.1667
  - 가구-인테리어(홈데코): 0.1667
  - 생활-건강(애완): 0.1667
  - 생활-건강(청소용품): 0.1654
  - 생활-건강(주방용품): 0.1503
  - 식품(김치): 0.0000
  - 식품(농산물): 0.0000
  - 식품(반찬): 0.0000
  - 생활-건강(안마용품): 0.0000
  - 생활-건강(욕실용품): 0.0000

군집 1 상위 선호 카테고리:
  - 패션의류: 0.4676
  - 생활-건강: 0.3324
  - 패션의류(여성언더웨어-잠옷): 0.2667
  - 패션의류(남성언더웨어-잠옷): 0.1516
  - 생활-건강(주방용품): 0.1333
  - 식품(가공식품): 0.1032
  - 식품: 0.1032
  - 생활-건강(건강관리용품): 0.0994
  - 패션의류(남성의류): 0.0968
  - 패션의류: 0.0968
  - 생활-건강(수납-정리용품): 0.0500
  - 패션의류(남성의류): 0.0496
  - 생활-건강(욕실용품): 0.0495
  - 식품(김치): 0.0000
  - 식품(과자): 0.0000

군집 2 상위 선호 카테고리:
  - 식품: 0.2303
  - 생활-건강: 0.2158
  - 화장품-미용: 0.1556
  - 생활-건강(주방용품): 0.1299
  - 디지털-가전: 0.0632
  - 식품(음료): 0.0631
  - 화장품-미용(스킨케어): 0.0608
  - 패션잡화: 0.0574
  - 생활-건강: 0.0425
  - 패션잡화(양말): 0.0393
  - 화장품-미용: 0.0387
  - 식품(가공식품): 0.0344
  - 패션잡화: 0.0322
  - 식품(건강식품): 0.0301
  - 디지털-가전: 0.0292

군집

In [67]:
df_num_sp.head()

,total_purchase_count,survival_prob,is_churn,current_shoji,frequency,recency,monetary,days_since_last_activity,가구-인테리어_1_1,디지털-가전_1_1,생활-건강_1_1,스포츠-레저_1_1,식품_1_1,출산-육아_1_1,패션의류_1_1,패션잡화_1_1,화장품-미용_1_1,가구-인테리어_1_2,디지털-가전_1_2,생활-건강_1_2,스포츠-레저_1_2,식품_1_2,출산-육아_1_2,패션의류_1_2,패션잡화_1_2,화장품-미용_1_2,가구-인테리어_1_3,디지털-가전_1_3,생활-건강_1_3,스포츠-레저_1_3,식품_1_3,출산-육아_1_3,패션의류_1_3,패션잡화_1_3,화장품-미용_1_3,가구-인테리어_1_4,디지털-가전_1_4,생활-건강_1_4,스포츠-레저_1_4,식품_1_4,출산-육아_1_4,패션의류_1_4,패션잡화_1_4,화장품-미용_1_4,가구-인테리어_1_5,디지털-가전_1_5,생활-건강_1_5,스포츠-레저_1_5,식품_1_5,출산-육아_1_5,패션의류_1_5,패션잡화_1_5,화장품-미용_1_5,가구-인테리어(DIY자재-용품)_2_1,가구-인테리어(거실가구)_2_1,가구-인테리어(수납가구)_2_1,가구-인테리어(수예)_2_1,가구-인테리어(인테리어소품)_2_1,가구-인테리어(침구단품)_2_1,가구-인테리어(카페트-러그)_2_1,가구-인테리어(홈데코)_2_1,디지털-가전(PC액세서리)_2_1,디지털-가전(계절가전)_2_1,디지털-가전(노트북)_2_1,디지털-가전(생활가전)_2_1,디지털-가전(음향가전)_2_1,디지털-가전(이미용가전)_2_1,디지털-가전(주방가전)_2_1,디지털-가전(주변기기)_2_1,디지털-가전(휴대폰악세서리)_2_1,생활-건강(건강관리용품)_2_1,생활-건강(건강측정용품)_2_1,생활-건강(공구)_2_1,생활-건강(구강위생용품)_2_1,생활-건강(문구-사무용품)_2_1,생활-건강(발건강용품)_2_1,생활-건강(생활용품)_2_1,생활-건강(세탁용품)_2_1,생활-건강(수납-정리용품)_2_1,생활-건강(안마용품)_2_1,생활-건강(애완)_2_1,생활-건강(온열-찜질용품)_2_1,생활-건강(욕실용품)_2_1,생활-건강(의료용품)_2_1,생활-건강(자동차용품)_2_1,생활-건강(주방용품)_2_1,생활-건강(청소용품)_2_1,스포츠-레저(골프)_2_1,스포츠-레저(기타스포츠용품)_2_1,스포츠-레저(등산)_2_1,스포츠-레저(스포츠액세서리)_2_1,스포츠-레저(캠핑)_2_1,스포츠-레저(헬스)_2_1,식품(가공식품)_2_1,식품(건강식품)_2_1,식품(과자)_2_1,식품(김치)_2_1,식품(냉동-간편조리식품)_2_1,식품(농산물)_2_1,식품(다이어트식품)_2_1,식품(반찬)_2_1,식품(수산)_2_1,식품(음료)_2_1,식품(축산)_2_1,출산-육아(물티슈)_2_1,출산-육아(위생-건강용품)_2_1,패션의류(남성언더웨어-잠옷)_2_1,패션의류(남성의류)_2_1,패션의류(여성언더웨어-잠옷)_2_1,패션의류(여성의류)_2_1,패션잡화(모자)_2_1,패션잡화(양말)_2_1,패션잡화(여성가방)_2_1,패션잡화(여성신발)_2_1,패션잡화(여행용가방-소품)_2_1,패션잡화(장갑)_2_1,패션잡화(패션소품)_2_1,패션잡화(헤어액세서리)_2_1,화장품-미용(남성화장품)_2_1,화장품-미용(마스크-팩)_2_1,화장품-미용(바디케어)_2_1,화장품-미용(베이스메이크업)_2_1,화장품-미용(뷰티소품)_2_1,화장품-미용(색조메이크업)_2_1,화장품-미용(선케어)_2_1,화장품-미용(스킨케어)_2_1,화장품-미용(클렌징)_2_1,화장품-미용(향수)_2_1,화장품-미용(헤어스타일링)_2_1,화장품-미용(헤어케어)_2_1,가구-인테리어(DIY자재-용품)_2_2,가구-인테리어(거실가구)_2_2,가구-인테리어(서재-사무용가구)_2_2,가구-인테리어(솜류)_2_2,가구-인테리어(수납가구)_2_2,가구-인테리어(인테리어소품)_2_2,가구-인테리어(주방가구)_2_2,가구-인테리어(침구단품)_2_2,가구-인테리어(카페트-러그)_2_2,가구-인테리어(홈데코)_2_2,디지털-가전(PC액세서리)_2_2,디지털-가전(계절가전)_2_2,디지털-가전(생활가전)_2_2,디지털-가전(영상가전)_2_2,디지털-가전(음향가전)_2_2,디지털-가전(이미용가전)_2_2,디지털-가전(주방가전)_2_2,디지털-가전(휴대폰악세서리)_2_2,생활-건강(건강관리용품)_2_2,생활-건강(건강측정용품)_2_2,생활-건강(공구)_2_2,생활-건강(구강위생용품)_2_2,생활-건강(문구-사무용품)_2_2,생활-건강(발건강용품)_2_2,생활-건강(생활용품)_2_2,생활-건강(세탁용품)_2_2,생활-건강(수납-정리용품)_2_2,생활-건강(안마용품)_2_2,생활-건강(애완)_2_2,생활-건강(욕실용품)_2_2,생활-건강(의료용품)_2_2,생활-건강(자동차용품)_2_2,생활-건강(재활운동용품)_2_2,생활-건강(주방용품)_2_2,생활-건강(청소용품)_2_2,스포츠-레저(골프)_2_2,스포츠-레저(기타스포츠용품)_2_2,스포츠-레저(등산)_2_2,스포츠-레저(배드민턴)_2_2,스포츠-레저(수영)_2_2,스포츠-레저(스포츠액세서리)_2_2,스포츠-레저(자전거)_2_2,스포츠-레저(캠핑)_2_2,식품(가공식품)_2_2,식품(건강식품)_2_2,식품(과자)_2_2,식품(김치)_2_2,식품(냉동-간편조리식품)_2_2,식품(농산물)_2_2,식품(다이어트식품)_2_2,식품(반찬)_2_2,식품(수산)_2_2,식품(아이스크림-빙수)_2_2,식품(음료)_2_2,식품(축산)_2_2,출산-육아(물티슈)_2_2,출산-육아(안전용퓸)_2_2,출산-육아(완구-매트)_2_2,출산-육아(외출용품)_2_2,출산-육아(위생-건강용품)_2_2,패션의류(남성언더웨어-잠옷)_2_2,패션의류(남성의류)_2_2,패션의류(여성언더웨어-잠옷)_2_2,패션의류(여성의류)_2_2,패션잡화(남성신발)_2_2,패션잡화(모자)_2_2,패션잡화(신발용품)_2_2,패션잡화(양말)_2_2,패션잡화(여성가방)_2_2,패션잡화(여성신발)_2_2,패션잡화(장갑)_2_2,패션잡화(주얼리)_2_2,패션잡화(지갑)_2_2,패션잡화(패션소품)_2_2,패션잡화(헤어액세서리)_2_2,화장품-미용(남성화장품)_2_2,화장품-미용(마스크-팩)_2_2,화장품-미용(바디케어)_2_2,화장품-미용(베이스메이크업)_2_2,화장품-미용(뷰티소품)_2_2,화장품-미용(색조메이크업)_2_2,화장품-미용(선케어)_2_2,화장품-미용(스킨케어)_2_2,화장품-미용(클렌징)_2_2,화장품-미용(향수)_2_2,화장품-미용(헤어스타일링)_2_2,화장품-미용(헤어케어)_2_2,가구-인테리어(DIY자재-용품)_2_3,가구-인테리어(거실가구)_2_3,가구-인테리어(서재-사무용가구)_2_3,가구-인테리어(침구단품)_2_3,가구-인테리어(카페트-러그)_2_3,가구-인테리어(홈데코)_2_3,디지털-가전(PC액세서리)_2_3,디지털-가전(계절가전)_2_3,디지털-가전(생활가전)_2_3,디지털-가전(음향가전)_2_3,디지털-가전(이미용가전)_2_3,디지털-가전(주방가전)_2_3,디지털-가전(휴대폰악세서리)_2_3,생활-건강(건강관리용품)_2_3,생활-건강(건강측정용품)_2_3,생활-건강(공구)_2_3,생활-건강(구강위생용품)_2_3,생활-건강(문구-사무용품)_2_3,생활-건강(발건강용품)_2_3,생활-건강(생활용품)_2_3,생활-건강(세탁용품)_2_3,생활-건강(수납-정리용품)_2_3,생활-건강(안마용품)_2_3,생활-건강(애완)_2_3,생활-건강(욕실용품)_2_3,생활-건강(의료용품)_2_3,생활-건강(자동차용품)_2_3,생활-건강(주방용품)_2_3,생활-건강(청소용품)_2_3,스포츠-레저(골프)_2_3,스포츠-레저(등산)_2_3,스포츠-레저(자전거)_2_3,스포츠-레저(캠핑)_2_3,식품(가공식품)_2_3,식품(건강식품)_2_3,식품(과자)_2_3,식품(김치)_2_3,식품(냉동-간편조리식품)_2_3,식품(농산물)_2_3,식품(다이어트식품)_2_3,식품(반찬)_2_3,식품(수산)_2_3,식품(아이스크림-빙수)_2_3,식품(음료)_2_3,식품(축산)_2_3,출산-육아(물티슈)_2_3,출산-육아(안전용퓸)_2_3,출